In [ ]:
!nvedia-smi

In [ ]:
# Transformers installation
! pip install transformers datasets
! pip install transformers datasets evaluate rouge_score

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
import huggingface_hub

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf")

In [ ]:
huggingface_hub.login(token = secret_value_0 ,write_permission =True )

In [ ]:
!pip install wandb

In [ ]:
from datasets import Dataset

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("wandb-key")


wandb.login(key=secret_value)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob

In [ ]:
# Use your fine tuning file
filename = "/kaggle/input/bert-cls-in-abs/IN-abs_CLS.xlsx"

df = pd.read_excel(filename,index_col=0)
df = df.reset_index(drop=True)  # Reset the index without creating a new column
df.rename(columns = {'data':'text', 'summary':'summary'}, inplace = True)
len(df)

In [ ]:
df.columns

In [ ]:
dataset = Dataset.from_pandas(df[['text', 'summary']])

In [ ]:
dataset

In [ ]:
dataset = dataset.train_test_split(test_size=0.0001)

In [ ]:
dataset

In [ ]:
# dataset["train"][0]

In [ ]:
# Loading Model and tokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
import pickle

# Assuming tokenized_dataset is your tokenized dataset
with open('tokenized_dataset.pkl', 'wb') as pkl_file:
    pickle.dump(tokenized_dataset, pkl_file)


Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
checkpoint = "facebook/bart-large"

In [ ]:
from transformers import DataCollatorForSeq2Seq


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=bart_model)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-cls-n",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=bart_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import os

# Define the directory where you want to save the model
output_directory = "model"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save the model, tokenizer, and configuration to the specified directory
bart_model.save_pretrained(output_directory)
tokenizer.save_pretrained(output_directory)



<Tip>

For a more in-depth example of how to finetune a model for summarization, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization-tf.ipynb).

</Tip>

In [ ]:
# text = dataset["test"]["text"]

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for summarization with your model, and pass your text to it:

In [ ]:
# # Load the saved model and tokenizer for testing
# model = AutoModelForSeq2SeqLM.from_pretrained(output_directory)
# tokenizer = AutoTokenizer.from_pretrained(output_directory)

# # Instantiate a pipeline for summarization with the saved model
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# # Generate the summary
# generated_summary = summarizer(text)

# # Print the generated summary
# print(generated_summary)


In [ ]:
!zip -r bart.zip /kaggle/working

In [ ]:
from IPython.display import FileLink
FileLink(r'bart.zip')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


# Define the name for your model on Hugging Face Hub
hub_model_name = "astro21/bart-cls_n"

# Save the model and tokenizer to the Hugging Face Model Hub
bart_model.push_to_hub(hub_model_name)
tokenizer.push_to_hub(hub_model_name)

# Once the above is done, you can also save the configuration for the model
bart_model.config.push_to_hub(hub_model_name)

# Commit your changes
bart_model.push_to_hub(hub_model_name, commit_message="Initial commit")

print(f"Model and tokenizer are now available on the Hugging Face Model Hub with the name: {hub_model_name}")
